In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import urllib.request
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/NLP/shopping.txt', sep = '\t', header = None)
print('총 샘플의 수 :',len(data))

총 샘플의 수 : 200000


In [ ]:
data.head()

,0,1
0,5,배공빠르고 굿
1,2,택배가 엉망이네용 저희집 밑에층에 말도없이 놔두고가고
2,5,아주좋아요 바지 정말 좋아서2개 더 구매했어요 이가격에 대박입니다. 바느질이 조금 ...
3,2,선물용으로 빨리 받아서 전달했어야 하는 상품이었는데 머그컵만 와서 당황했습니다. 전...
4,5,민트색상 예뻐요. 옆 손잡이는 거는 용도로도 사용되네요 ㅎㅎ


In [ ]:
data['label'] = np.select([data[0] > 3], [1], default=0) # 3 안되는 수 0으로.

In [ ]:
data.head()

,0,1,label
0,5,배공빠르고 굿,1
1,2,택배가 엉망이네용 저희집 밑에층에 말도없이 놔두고가고,0
2,5,아주좋아요 바지 정말 좋아서2개 더 구매했어요 이가격에 대박입니다. 바느질이 조금 ...,1
3,2,선물용으로 빨리 받아서 전달했어야 하는 상품이었는데 머그컵만 와서 당황했습니다. 전...,0
4,5,민트색상 예뻐요. 옆 손잡이는 거는 용도로도 사용되네요 ㅎㅎ,1


In [ ]:
data.drop_duplicates(subset=[1], inplace=True)
len(data)

199908

In [ ]:
data.isnull().sum()

0        0
1        0
label    0
dtype: int64

In [ ]:
train_data, test_data = train_test_split(data, test_size = 0.2, random_state = 42)

In [ ]:
print(len(train_data))
print(len(test_data))

159926
39982


In [ ]:
train_data[1] = train_data[1].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣\s]","") #공백문자 제외.
train_data[1].replace('', np.nan, inplace=True)

<ipython-input-14-7543f700a5e0>:1: FutureWarning: The default value of regex will change from True to False in a future version.
  train_data[1] = train_data[1].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣\s]","") #공백문자 제외.


In [ ]:
train_data.head()

,0,1,label
132466,1,바디버든 줄이려고 써보려고 샀는데 음용할수잇는 물로 헹궈야되면 왜 식기세척기 쓰나요,0
188461,2,시중에 판매하는 건새우랑 다르게 속살이 고루분포된게아니라 속이비어보이고 속살이 한쪽...,0
124757,4,깔끔하니 이쁘네요 좋습니다,1
50357,2,사이즈미스 남자가 쓰기엔 너무작네요,0
137655,4,좋은 상품 빠르고 안전한 배송 감사합니다,1


In [ ]:
train_data.isnull().sum()

0        0
1        0
label    0
dtype: int64

In [ ]:
test_data[1] = data[1].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","") # 정규 표현식 수행
test_data[1].replace('', np.nan, inplace=True)

<ipython-input-16-38e06aaf39e4>:1: FutureWarning: The default value of regex will change from True to False in a future version.
  test_data[1] = data[1].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","") # 정규 표현식 수행


In [ ]:
test_data.head()

,0,1,label
193242,1,너무 낮고 솜도 적고 실망스럽습니다,0
125080,1,피부에 뾰루지가 많이 올라와요,0
122750,5,배송도 빠르네요 가격대비 좋은것 같아요 첨에는 힘들어하나 조금 지나니 잘 하네요,1
72927,5,재구매입니다 핏도 좋고 착용감도 좋습니다,1
83890,1,파손제품 온거 출장같다 오늘 받았는데 현재상황 장난하시는지 택배회사 항의하세요,0


In [ ]:
test_data.isnull().sum()

0        0
1        0
label    0
dtype: int64

In [ ]:
!pip install konlpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 41.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.3/465.3 kB 42.6 MB/s eta 0:00:00


In [ ]:
from konlpy.tag import Okt

In [ ]:
okt = Okt()

In [ ]:
stopwords = ['도', '는', '다', '의', '가', '이', '은', '한', '에', '하', '고', '을', '를', '인', '듯', '과', '와', '네', '들', '듯', '지', '임', '게']

In [ ]:
train_data['tokenized'] = train_data[1].apply(okt.morphs)
train_data['tokenized'] = train_data[1].apply(lambda x: [item for item in x if item not in stopwords])

In [ ]:
test_data['tokenized'] = test_data[1].apply(okt.morphs) # 형태소 분리.
test_data['tokenized'] = test_data[1].apply(lambda x: [item for item in x if item not in stopwords])

In [ ]:
X_train = train_data['tokenized'].values
y_train = train_data['label'].values
X_test= test_data['tokenized'].values
y_test = test_data['label'].values

In [ ]:
tokenizer = Tokenizer() ##
tokenizer.fit_on_texts(X_train)

In [ ]:
total_cnt = len(tokenizer.word_index)

In [ ]:
vocab_size = total_cnt + 1

In [ ]:
tokenizer = Tokenizer(vocab_size, oov_token = 'OOV')
tokenizer.fit_on_texts(X_train)
X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

In [ ]:
X_train = pad_sequences(X_train, maxlen = 40)
X_test = pad_sequences(X_test, maxlen = 40)

In [ ]:
w2i = tokenizer.word_index

In [ ]:
vocab_size = len(w2i) + 1

In [ ]:
from tensorflow.keras.layers import SimpleRNN, Embedding, Dense, LSTM
from tensorflow.keras.models import Sequential

In [ ]:
model = Sequential()

In [ ]:
model.add(Embedding(vocab_size, 32)) #LSTM 올라가기 전에 각 output값 싹다 32차원 맞춰준다는 말.
model.add(SimpleRNN(100, return_sequences = True))
model.add(SimpleRNN(50)) #
model.add(Dense(1, activation = 'sigmoid'))

In [ ]:
#model.compile(optimizer = 'rmsprop', loss = 'binary_crossentropy', metrics = ['accuracy'])
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [ ]:
history = model.fit(X_train, y_train, epochs = 5, batch_size = 32, validation_split = 0.2)

Epoch 1/5
3999/3999 [==============================] - 128s 31ms/step - loss: 0.3911 - accuracy: 0.8337 - val_loss: 0.3569 - val_accuracy: 0.8557
Epoch 2/5
3999/3999 [==============================] - 122s 31ms/step - loss: 0.3640 - accuracy: 0.8540 - val_loss: 0.3727 - val_accuracy: 0.8496
Epoch 3/5
3999/3999 [==============================] - 124s 31ms/step - loss: 0.3435 - accuracy: 0.8653 - val_loss: 0.3462 - val_accuracy: 0.8628
Epoch 4/5
3999/3999 [==============================] - 125s 31ms/step - loss: 0.3387 - accuracy: 0.8669 - val_loss: 0.3385 - val_accuracy: 0.8664
Epoch 5/5
3999/3999 [==============================] - 121s 30ms/step - loss: 0.3303 - accuracy: 0.8708 - val_loss: 0.3278 - val_accuracy: 0.8683


In [ ]:
xtest_padded = pad_sequences(X_test, maxlen = 48)

In [ ]:
model.evaluate(xtest_padded, y_test)[1]

1250/1250 [==============================] - 13s 10ms/step - loss: 0.3335 - accuracy: 0.8670


0.866965115070343